In [18]:
import sqlite3
import pandas as pd
from random import sample

In [7]:
conn = sqlite3.connect('../data/trip_data.db')

In [14]:
q_license = """
SELECT distinct(hack_license) as UNIQ_LIC
FROM trip
"""

unique_license = pd.read_sql(q_license, conn)

In [81]:
LIC_SAMPLE_NUM = 10
sampled_drivers = sample(unique_license.UNIQ_LIC.tolist(), LIC_SAMPLE_NUM)

In [82]:
q_sample = """
SELECT *, 
        lag(pickup_datetime, -1, NULL) over (partition by hack_license order by pickup_datetime) as pickup_datetime_next,
        lag(dropoff_datetime, -1, NULL) over (partition by hack_license order by pickup_datetime) as dropoff_datetime_next
    FROM trip
    WHERE hack_license in {0}
ORDER BY hack_license ASC, pickup_datetime ASC, dropoff_datetime ASC
"""

sampled_trip = pd.read_sql(q_sample.format(tuple(sampled_drivers)), conn)

In [83]:
sampled_trip.head()

,medallion,hack_license,vendor_id,rate_code,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,pickup_datetime_next,dropoff_datetime_next
0,2013002073,2013002070,CMT,1,2013-06-01 00:04:23,2013-06-01 00:18:32,3,848,4.9,-73.977119,40.785114,-74.010185,40.729843,2013-06-01 00:21:29,2013-06-01 00:29:42
1,2013002073,2013002070,CMT,1,2013-06-01 00:21:29,2013-06-01 00:29:42,3,492,1.8,-74.007668,40.733086,-73.977966,40.725433,2013-06-01 00:31:06,2013-06-01 00:38:40
2,2013002073,2013002070,CMT,1,2013-06-01 00:31:06,2013-06-01 00:38:40,3,453,1.3,-73.978233,40.724945,-73.998802,40.728340,2013-06-01 00:40:02,2013-06-01 00:53:52
3,2013002073,2013002070,CMT,1,2013-06-01 00:40:02,2013-06-01 00:53:52,1,829,2.8,-73.999062,40.728035,-73.997650,40.756626,2013-06-01 01:01:57,2013-06-01 01:15:38
4,2013002073,2013002070,CMT,1,2013-06-01 01:01:57,2013-06-01 01:15:38,1,820,3.7,-74.007233,40.744305,-73.978516,40.788982,2013-06-01 01:37:07,2013-06-01 01:40:35


In [86]:
date_cols = ['pickup_datetime', 'dropoff_datetime', 'pickup_datetime_next', 'dropoff_datetime_next']

for c in date_cols:
    sampled_trip[c] = pd.to_datetime(sampled_trip[c])
sampled_trip['delta'] = (sampled_trip['pickup_datetime_next'] - sampled_trip['dropoff_datetime']) / pd.Timedelta(hours=1)

In [107]:
k=5

## If delta > k, the trip is the last drop off of the shift
## And the next trip of the driver is the first pickup (shifted)
sampled_trip['last_dropoff'] = [1 if d > k else 0 for d in sampled_trip['delta']]
sampled_trip['first_pickup'] = sampled_trip.groupby('hack_license')['last_dropoff'].shift(1)
sampled_trip.dropna(subset=['pickup_datetime_next', 'dropoff_datetime_next'], inplace=True) 
sampled_trip['first_pickup'].fillna(0, inplace=True)

In [108]:
# sampled_trip.loc[sampled_trip['first_pickup'] == 1]
sampled_trip

,medallion,hack_license,vendor_id,rate_code,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,pickup_datetime_next,dropoff_datetime_next,delta,first_pickup,last_dropoff
0,2013002073,2013002070,CMT,1,2013-06-01 00:04:23,2013-06-01 00:18:32,3,848,4.9,-73.977119,40.785114,-74.010185,40.729843,2013-06-01 00:21:29,2013-06-01 00:29:42,0.049167,0.0,0
1,2013002073,2013002070,CMT,1,2013-06-01 00:21:29,2013-06-01 00:29:42,3,492,1.8,-74.007668,40.733086,-73.977966,40.725433,2013-06-01 00:31:06,2013-06-01 00:38:40,0.023333,0.0,0
2,2013002073,2013002070,CMT,1,2013-06-01 00:31:06,2013-06-01 00:38:40,3,453,1.3,-73.978233,40.724945,-73.998802,40.728340,2013-06-01 00:40:02,2013-06-01 00:53:52,0.022778,0.0,0
3,2013002073,2013002070,CMT,1,2013-06-01 00:40:02,2013-06-01 00:53:52,1,829,2.8,-73.999062,40.728035,-73.997650,40.756626,2013-06-01 01:01:57,2013-06-01 01:15:38,0.134722,0.0,0
4,2013002073,2013002070,CMT,1,2013-06-01 01:01:57,2013-06-01 01:15:38,1,820,3.7,-74.007233,40.744305,-73.978516,40.788982,2013-06-01 01:37:07,2013-06-01 01:40:35,0.358056,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4811,2013010712,2013026394,CMT,1,2013-06-30 22:24:42,2013-06-30 22:34:05,1,563,1.2,-74.002045,40.719261,-73.987503,40.722393,2013-06-30 22:53:49,2013-06-30 23:03:37,0.328889,0.0,0
4812,2013010712,2013026394,CMT,1,2013-06-30 22:53:49,2013-06-30 23:03:37,1,588,1.7,-73.990654,40.723606,-73.998535,40.740253,2013-06-30 23:11:08,2013-06-30 23:22:26,0.125278,0.0,0
4813,2013010712,2013026394,CMT,1,2013-06-30 23:11:08,2013-06-30 23:22:26,1,677,4.3,-73.993355,40.752659,-73.960785,40.801579,2013-06-30 23:30:04,2013-06-30 23:44:25,0.127222,0.0,0
4814,2013010712,2013026394,CMT,1,2013-06-30 23:30:04,2013-06-30 23:44:25,1,861,4.6,-73.960419,40.797356,-74.000046,40.743534,2013-06-30 23:47:38,2013-06-30 23:54:25,0.053611,0.0,0


In [111]:
## Extract hour and day
sampled_trip['dropoff_hour'] = sampled_trip.dropoff_datetime.dt.hour
sampled_trip['pickup_hour'] = sampled_trip.pickup_datetime.dt.hour
sampled_trip['dropoff_day'] = sampled_trip.dropoff_datetime.dt.date
sampled_trip['pickup_day'] = sampled_trip.pickup_datetime.dt.date
sampled_trip['pickup_weekday'] = (sampled_trip.loc[:,'pickup_datetime'].dt.dayofweek // 5 == 1).astype(int)


In [112]:
sampled_trip.head()

,medallion,hack_license,vendor_id,rate_code,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,...,dropoff_datetime_next,delta,first_pickup,last_dropoff,dropoff_hour,pickup_hour,dropoff_day,pickup_day,weekday,pickup_weekday
0,2013002073,2013002070,CMT,1,2013-06-01 00:04:23,2013-06-01 00:18:32,3,848,4.9,-73.977119,...,2013-06-01 00:29:42,0.049167,0.0,0,0,0,2013-06-01,2013-06-01,1,1
1,2013002073,2013002070,CMT,1,2013-06-01 00:21:29,2013-06-01 00:29:42,3,492,1.8,-74.007668,...,2013-06-01 00:38:40,0.023333,0.0,0,0,0,2013-06-01,2013-06-01,1,1
2,2013002073,2013002070,CMT,1,2013-06-01 00:31:06,2013-06-01 00:38:40,3,453,1.3,-73.978233,...,2013-06-01 00:53:52,0.022778,0.0,0,0,0,2013-06-01,2013-06-01,1,1
3,2013002073,2013002070,CMT,1,2013-06-01 00:40:02,2013-06-01 00:53:52,1,829,2.8,-73.999062,...,2013-06-01 01:15:38,0.134722,0.0,0,0,0,2013-06-01,2013-06-01,1,1
4,2013002073,2013002070,CMT,1,2013-06-01 01:01:57,2013-06-01 01:15:38,1,820,3.7,-74.007233,...,2013-06-01 01:40:35,0.358056,0.0,0,1,1,2013-06-01,2013-06-01,1,1


In [ ]:
# calculate percentage of pickup and dropoff in each hour, averaged across date
pickup_per = pick_drop.groupby(['pickup_day_next','weekday','pickup_hour_next'])['pickup_hour_next'].count()
# percentage = # pickup in each hour / # pickup in the day
pickup_per = pickup_per.groupby(level=0).apply(lambda x: x / x.sum() * 100) 
pickup_per = pd.DataFrame(pickup_per).rename(columns={'pickup_hour_next':'pickup_percent'}).reset_index().\
                                        rename(columns={'pickup_day_next':'date',
                                                        'pickup_hour_next':'hour'})

dropoff_per = pick_drop.groupby(['dropoff_day_now','weekday','dropoff_hour_now'])['dropoff_hour_now'].count()
dropoff_per = dropoff_per.groupby(level=0).apply(lambda x: x / x.sum() * 100) 
dropoff_per = pd.DataFrame(dropoff_per).rename(columns={'dropoff_hour_now':'dropoff_percent'}).reset_index().\
                                        rename(columns={'dropoff_day_now':'date',
                                                        'dropoff_hour_now':'hour'})
pick_drop_per = pickup_per.merge(dropoff_per)
pick_drop_per.head()